<a href="https://colab.research.google.com/github/Sarehj/Database-Design-I/blob/main/DatabaseDesignI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 40.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


In [3]:
pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.3 MB/s eta 0:00:00


In [4]:
pip install sshtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 52.0 MB/s eta 0:00:00


In [ ]:
import pymysql
from sshtunnel import SSHTunnelForwarder
tunnel = SSHTunnelForwarder(
('fries.it.uu.se', 22),# do not change 22, this is a portal
ssh_username = 'saja2454', # use your Studium username
ssh_password = 'S.j.A.1377', # use your Studium password
remote_bind_address=('127.0.0.1', 3306)
)
tunnel.start()
mydb = pymysql.connect(
host='127.0.0.1',
user='ht23_1_group_39',
password='pasSWd_39',
port=tunnel.local_bind_port,
db = 'ht23_1_project_group_39'
)
mycursor = mydb.cursor ()
mycursor.execute ("SHOW TABLES")
for x in mycursor:
  print (x)
mydb.close()


('Customer',)
('CustomerAddress',)
('Department',)
('Homepage',)
('Order',)
('Product',)
('ProductQuantity',)
('Related',)
('Review',)


In [12]:
## Task 1 and 2

import pymysql
from sshtunnel import SSHTunnelForwarder
tunnel = SSHTunnelForwarder(
('fries.it.uu.se', 22),
ssh_username = 'saja2454',
ssh_password = 'S.j.A.1377',
remote_bind_address=('127.0.0.1', 3306)
)
tunnel.start()
conn = pymysql.connect(
host='127.0.0.1',
user='ht23_1_group_39',
password='pasSWd_39',
port=tunnel.local_bind_port,
db = 'ht23_1_project_group_39'
)


cursor = conn.cursor()

# Task 1: List products or child departments of a given department
def list_products_or_child_departments(department_id):
    query = "SELECT Title, DepartmentID FROM Department WHERE ParentDepartmentID = %s"
    cursor.execute(query, (department_id,))
    child_departments = cursor.fetchall()

    if child_departments:
        print("Child Departments:")
        for department in child_departments:
            print(f"ID: {department[1]}, Title: {department[0]}")
    else:
        query = "SELECT P.ProductID, P.ProductTitle, P.RetailPriceNoVAT, (P.RetailPriceNoVAT - (P.RetailPriceNoVAT * P.DiscountPercentage / 100)) AS DiscountedPrice FROM Product P WHERE DepartmentID = %s"
        cursor.execute(query, (department_id,))
        products = cursor.fetchall()

        if products:
            print("Products in the Department:")
            for product in products:
                print(f"ID: {product[0]}, Title: {product[1]}, Discounted Price: ${product[3]:.2f}")
        else:
            print("No child departments or products found for the given department ID.")




# Task 2: Change the discount percentage of a product
def change_discount(product_id):

    query = "SELECT ProductTitle, DiscountPercentage FROM Product WHERE ProductID = %s"
    cursor.execute(query, (product_id,))
    result = cursor.fetchone()

    if result:
        current_discount = result[1]
        product_name = result[0]

        print(f"Current Discount for Product {product_name} (ID: {product_id}): {current_discount}%")

        new_discount = input("Enter the new discount percentage: ")
        query = "UPDATE Product SET DiscountPercentage = %s WHERE ProductID = %s"
        cursor.execute(query, (new_discount, product_id))
        conn.commit()
        print(f"Discount for Product {product_name} (ID: {product_id}) updated to {new_discount}%")

    else:
        print(f"No product found with Product ID {product_id}.")


# Take user input for the tasks
while True:
    print("\nChoose a task:")
    print("1. List products or child departments of a department")
    print("2. Change the discount of a product")
    print("0. Exit")
    choice = input("Enter the task number (0/1/2): ")

    if choice == "1":
        department_id = input("Enter the department ID: ")
        list_products_or_child_departments(department_id)
    elif choice == "2":
        product_id = input("Enter the product ID: ")
        change_discount(product_id)
    elif choice == "0":
        break
    else:
        print("Invalid choice. Please enter a valid task number.")

# Close the cursor and connection
cursor.close()
conn.close()



Choose a task:
1. List products or child departments of a department
2. Change the discount of a product
0. Exit
Enter the task number (0/1/2): 2
Enter the product ID: 3
Current Discount for Product CasualChic Men's Shirt (ID: 3): 15.00%
Enter the new discount percentage: 35
Discount for Product CasualChic Men's Shirt (ID: 3) updated to 35%

Choose a task:
1. List products or child departments of a department
2. Change the discount of a product
0. Exit
Enter the task number (0/1/2): 0
